In [79]:
# !pip install hsclient
targets = ["www.hydroshare.org"]
meta_run_number = 0 # manually change for first run/ second
if meta_run_number == 0:
    meta_runs = [{}, {}] # pre and post
clients = {}
filename = "local-build-discover.sh"
diffs= {}

In [80]:
from hsclient import HydroShare
for target in targets:
    hs = HydroShare(host=target, port=443, protocol='https')
    hs.sign_in()
    hs.resources = []
    diffs[target] = []
    clients[target] = hs

In [105]:
from time import time
from functools import wraps
import numpy as np
import scipy
from copy import deepcopy

def timer_func(func): 
    @wraps(func)
    def wrap_func(*args, **kwargs): 
        t1 = time() 
        result = func(*args, **kwargs)
        t2 = time()
        target = kwargs['target']
        diff = t2-t1
        diffs[target].append(diff)
        print(f'Function {func.__name__!r} executed in {(diff):.4f}s') 
        return result 
    return wrap_func


def create_res(hs, time=True, target=None):

    @timer_func
    def create_res_time(hs=None, target=None):
        resource = hs.create()
        hs.resources.append(resource)
    
    if time:
        create_res_time(hs=hs, target=target)
    else:
        resource = hs.create()
        hs.resources.append(resource)

@timer_func
def delete_resources(hs=None, target=None):
    for res in hs.resources:
        res.delete()

@timer_func
def download_all_resources(hs=None, target=None):
    for res in hs.resources:
        try:
            res.download()
        except Exception as e:
            print(e)

def download_single_resource(hs=None, target=None):
    create_res(hs, False)
    res = hs.resources[-1]
    res.file_upload(filename)

    @timer_func
    def download_single_resource_inner(hs=res, target=None):
        res.download()

    download_single_resource_inner(hs=res, target=target)

def delete_single_resource(hs=None, target=None):
    create_res(hs, False)
    res = hs.resources[-1]
    res.file_upload(filename)

    @timer_func
    def delete_single_resource_inner(hs=res, target=None):
        res.delete()

    delete_single_resource_inner(hs=res, target=target)

def upload_file(hs, target=None):
    create_res(hs, False)
    res = hs.resources[-1]

    @timer_func
    def upload_file_inner(hs=res, target=None):
        res.file_upload(filename)

    upload_file_inner(hs=res, target=target)


def create_dir_upload_file(hs=None, target=None):
    create_res(hs, False)
    res = hs.resources[-1]

    @timer_func
    def create_dir_upload_file_inner(hs=res, target=None):
        res.folder_create('New_Folder')
        res.file_upload(filename, destination_path='New_Folder')
    create_dir_upload_file_inner(hs=res, target=target)

def upload_and_movefile(hs=None, target=None):
    create_res(hs, False)
    res = hs.resources[-1]
    
    @timer_func
    def upload_and_movefile_inner(hs=res, target=None):
        res.folder_create('New_Folder')
        res.file_upload(filename, destination_path='New_Folder')
        file = res.file(path=f"New_Folder/{filename}]")
        res.file_rename(file, filename)
    upload_and_movefile_inner(hs=res, target=target)

def download_single_file(hs=None, target=None):
    create_res(hs, False)
    res = hs.resources[-1]
    res.file_upload(filename)

    @timer_func
    def download_single_file_inner(hs=res, target=target):
        file = res.file(path=filename)
        res.file_download(file)
    download_single_file_inner(hs=res, target=target)

def delete_single_file(hs=None, target=None):
    create_res(hs, False)
    res = hs.resources[-1]
    res.file_upload(filename)

    @timer_func
    def delete_single_file_inner(hs=res, target=None):
        file = res.file(path=filename)
        res.file_delete(file)
    delete_single_file_inner(hs=res, target=target)
        

def run_comparisons(functions_to_run, runs=1):
    for funct in functions_to_run:
        print(f"********* Start {funct.__name__!r} *********")
        for i in range(runs):
            for target in targets:
                client = clients[target]
                print(f"Starting run {i} of {funct.__name__} on {target}...")
                funct(hs=client, target=target)
                print(diffs[target])
        print("\n\n")
        # Store the diffs for later
        meta_runs[meta_run_number][funct.__name__] = deepcopy(diffs)
        for target in targets:
            diffs[target].clear()

# def run_stats():
#     for meta_run in meta_runs:
#         for target in targets:
#             client = clients[target]
#             print(f"Average time {funct.__name__!r} on {target}: {sum(diffs[target])/len(diffs[target])}")
#             print(f"Devation for {funct.__name__!r} on {target}: {np.std(diffs[target])}")
#             # https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html
#         result = scipy.stats.ttest_ind(diffs[targets[0]], diffs[targets[1]])
#         print(result)
#         print("\n\n")
#         for target in targets:
#             diffs[target].clear()

def clear_resources():
    for target in targets:
        client = clients[target]
        client.resources = []
        diffs[target].clear()
  

In [106]:
clear_resources()
run_comparisons([download_single_resource, create_dir_upload_file, delete_single_file ], runs=3)
run_comparisons([delete_resources], runs=1)

********* Start 'download_single_resource' *********
Starting run 0 of download_single_resource on www.hydroshare.org...
Function 'download_single_resource_inner' executed in 19.6961s
[19.69610905647278]
Starting run 1 of download_single_resource on www.hydroshare.org...
Function 'download_single_resource_inner' executed in 19.6573s
[19.69610905647278, 19.657349824905396]
Starting run 2 of download_single_resource on www.hydroshare.org...
Function 'download_single_resource_inner' executed in 19.3837s
[19.69610905647278, 19.657349824905396, 19.383678197860718]



********* Start 'create_dir_upload_file' *********
Starting run 0 of create_dir_upload_file on www.hydroshare.org...
Function 'create_dir_upload_file_inner' executed in 10.0985s
[10.0984628200531]
Starting run 1 of create_dir_upload_file on www.hydroshare.org...
Function 'create_dir_upload_file_inner' executed in 10.6771s
[10.0984628200531, 10.677123785018921]
Starting run 2 of create_dir_upload_file on www.hydroshare.org...
Fu

In [108]:
clear_resources()
run_comparisons([upload_file], runs=10)

********* Start 'upload_file' *********
Starting run 0 of upload_file on www.hydroshare.org...
Function 'upload_file_inner' executed in 2.8220s
[2.82197904586792]
Starting run 1 of upload_file on www.hydroshare.org...
Function 'upload_file_inner' executed in 2.8427s
[2.82197904586792, 2.8427178859710693]
Starting run 2 of upload_file on www.hydroshare.org...
Function 'upload_file_inner' executed in 2.9220s
[2.82197904586792, 2.8427178859710693, 2.9220480918884277]
Starting run 3 of upload_file on www.hydroshare.org...
Function 'upload_file_inner' executed in 2.8254s
[2.82197904586792, 2.8427178859710693, 2.9220480918884277, 2.825402021408081]
Starting run 4 of upload_file on www.hydroshare.org...
Function 'upload_file_inner' executed in 2.8815s
[2.82197904586792, 2.8427178859710693, 2.9220480918884277, 2.825402021408081, 2.881467819213867]
Starting run 5 of upload_file on www.hydroshare.org...
Function 'upload_file_inner' executed in 2.8290s
[2.82197904586792, 2.8427178859710693, 2.922

In [110]:
clear_resources()
run_comparisons([create_res, download_all_resources, delete_single_resource], runs=5)

********* Start 'create_res' *********
Starting run 0 of create_res on www.hydroshare.org...
Function 'create_res_time' executed in 15.4449s
[15.444867134094238]
Starting run 1 of create_res on www.hydroshare.org...
Function 'create_res_time' executed in 13.9518s
[15.444867134094238, 13.95179295539856]
Starting run 2 of create_res on www.hydroshare.org...
Function 'create_res_time' executed in 13.6506s
[15.444867134094238, 13.95179295539856, 13.65056300163269]
Starting run 3 of create_res on www.hydroshare.org...
Function 'create_res_time' executed in 14.3593s
[15.444867134094238, 13.95179295539856, 13.65056300163269, 14.359339952468872]
Starting run 4 of create_res on www.hydroshare.org...
Function 'create_res_time' executed in 13.7653s
[15.444867134094238, 13.95179295539856, 13.65056300163269, 14.359339952468872, 13.765339136123657]



********* Start 'download_all_resources' *********
Starting run 0 of download_all_resources on www.hydroshare.org...
Function 'download_all_resources'

In [ ]:
import pickle
file = open('pickle', 'ab')
    
# source, destination
pickle.dump(meta_runs, file)                    
file.close()

In [ ]:
dbfile = open('pickle', 'rb')    
meta_runs = pickle.load(dbfile)
dbfile.close()


In [ ]:
# TODO: these ones still seem not to work...
#clear_resources()
#run_comparisons([upload_and_movefile, download_single_file ], runs=2)